In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install gliner

  Using cached gliner-0.2.13-py3-none-any.whl.metadata (7.3 kB)
  Using cached onnxruntime-1.19.2-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 75.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 4.6 MB/s eta 0:00:00


In [6]:
# !pip install gliner-spacy

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [8]:
from gliner import GLiNER

In [9]:
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")
# ner_model = gliner.load_model('gliner')  # Load the GLiNER model

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/781M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/781M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [79]:
label_masks = {"Person" : "PERSON", "Organization":"ORGANIZATION", "Location" : "LOCATION", "Date": "DATE", "Event":"EVENT", "Monetary Value":"MONEY", "Product":"PRODUCT","Ordinal Number": "ORDINAL", "Percentage": "PERCENTAGE"} # Adjust the labels as necessary

In [80]:
labels = list(label_masks.keys())
labels

['Person',
 'Organization',
 'Location',
 'Date',
 'Event',
 'Monetary Value',
 'Product',
 'Ordinal Number',
 'Percentage']

In [81]:
root= "/kaggle/input/nlp-nifty50/Nifty50_news_data(2020Jan_2024April) (1).csv"
df= pd.read_csv(root)

In [82]:
df

,Unnamed: 0,datePublished,company,symbol,headline,description,articleBody,tags,author,url
0,0,2020-01-16 19:56:06+05:30,Adani Enterprises Ltd.,ADANIENT,"CBI books Adani Enterprises, 3 ex-NCCF officia...",The CBI alleged that acts of &amp;quot;omissio...,The CBI has booked Ahmedabad-based Adani Enter...,"Adani Enterprises, Business, CBI, India, Marke...","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/business/cb...
1,1,2020-01-20 11:30:07+05:30,Adani Enterprises Ltd.,ADANIENT,Adani Enterprises denies wrongdoing amid alleg...,The firm was replying to the BSE in response t...,Adani Enterprises on Monday said that it has n...,"Adani Enterprises, Business, CBI, coal supply,...","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/business/ad...
2,2,2020-02-04 15:26:12+05:30,Power Grid Corporation of India Ltd.,POWERGRID,Buy Power Grid Corporation of India; target of...,Prabhudas Lilladher is bullish on Power Grid C...,Prabhudas Lilladher's research report on Power...,"Buy, Power Grid Corporation of India, Prabhuda...","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/recommendat...
3,3,2020-02-05 17:20:09+05:30,Adani Enterprises Ltd.,ADANIENT,Adani Enterprises net profit surges to Rs 383 ...,"The company's expenses were Rs 10,635.16 crore...",Adani Enterprises Ltd (AEL) on Wednesday said ...,"Adani Enterprises Ltd, Business, Results","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/business/ad...
4,4,2020-02-10 18:05:05+05:30,Grasim Industries Ltd.,GRASIM,Grasim Industries Q3 net profit down 9.4% to R...,Total income during the quarter under review f...,Aditya Birla Group firm Grasim Industries Ltd ...,"Business, Grasim Industries Ltd., Results","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/business/gr...
...,...,...,...,...,...,...,...,...,...,...
2714,2714,2024-04-23 12:22:08+05:30,Wipro Ltd.,WIPRO,Reduce Wipro; target of Rs 410: ICICI Securities,ICICI Securities is bullish on Wipro has recom...,ICICI Securitie's research report on Wipro Wip...,"ICICI Securities, Recommendations, Reduce, Wipro","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/recommendat...
2715,2715,2024-04-23 13:41:02+05:30,HDFC Bank Ltd.,HDFCBANK,"Buy HDFC Bank; target of Rs 1,850: ICICI Secur...",ICICI Securities is bullish on HDFC Bank has r...,ICICI Securitie's research report on HDFC Bank...,"Buy, HDFC Bank, ICICI Securities, Recommendations","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/recommendat...
2716,2716,2024-04-23 14:47:18+05:30,Larsen & Toubro Ltd.,LT,L&amp;amp;T manufactures hydrotreating reactor...,The reactor has been dispatched for Mexico fro...,Engineering and construction conglomerate Lars...,"Business, Companies, Larsen & Toubro, Mexico","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/business/lt...
2717,2717,2024-04-23 16:05:45+05:30,Hindustan Unilever Ltd.,HINDUNILVR,"Hindustan Unilever Q4 Preview: Price cuts, slo...",The hike in royalty payments to parent Unileve...,Hindustan Unilever (HUL) is expected to report...,"Business, Hindustan Unilever, Result Poll","{'@type': 'Person', 'url': 'https://www.moneyc...",https://www.moneycontrol.com//news/business/hi...


In [83]:
def mask_entities_gliner(text):
    entities = model.predict_entities(text, labels,label_masks, threshold=0.5)
    masked_text = text
    # Replace each entity with its corresponding label in square brackets
    for entity in entities:
        masked_text = masked_text.replace(entity['text'], f'{label_masks[entity["label"]]}')
    return masked_text

In [84]:
def mask_headline_with_gliner(row):
    headline = row['headline']
#     tags = row['tags']
    
    # Combine tags with headline for better NER context
#     combined_text = f"{tags} {headline}"
    
    # Mask entities in the combined text using GLiNER
    masked_headline = mask_entities_gliner(headline)
    
    # Return the masked headline
    return masked_headline

In [85]:
sample_headline = df["headline"].values[0]
sample_tags= df["tags"].values[0]
sample_headline, sample_tags

('CBI books Adani Enterprises, 3 ex-NCCF officials for alleged irregularities in coal supply contract',
 'Adani Enterprises, Business, CBI, India, Market news')

In [86]:
mask_entities_gliner(sample_headline)

'ORGANIZATION books ORGANIZATION, 3 PERSON for alleged irregularities in EVENT'

In [87]:
for i in range(5):
    print(i+1)
    sample_headline= df["headline"].values[i]
    sample_tags= df["tags"].values[i]
    print(sample_headline)
    print(sample_tags)
    print(mask_entities_gliner(sample_headline))

1
CBI books Adani Enterprises, 3 ex-NCCF officials for alleged irregularities in coal supply contract
Adani Enterprises, Business, CBI, India, Market news
ORGANIZATION books ORGANIZATION, 3 PERSON for alleged irregularities in EVENT
2
Adani Enterprises denies wrongdoing amid allegations of coal supply contract irregularities
Adani Enterprises, Business, CBI, coal supply, Companies
ORGANIZATION denies wrongdoing amid allegations of EVENT
3
Buy Power Grid Corporation of India; target of Rs 224: Prabhudas Lilladher
Buy, Power Grid Corporation of India, Prabhudas Lilladher, Recommendations
Buy ORGANIZATION; target of MONEY: PERSON
4
Adani Enterprises net profit surges to Rs 383 cr in December quarter
Adani Enterprises Ltd, Business, Results
ORGANIZATION net profit surges to MONEY in DATE
5
Grasim Industries Q3 net profit down 9.4% to Rs 1,039.91cr
Business, Grasim Industries Ltd., Results
ORGANIZATION Q3 net profit down PERCENTAGE to MONEY.MONEY


In [88]:
len(df["headline"].values)

2719

In [89]:
masked_headlines = []

# Get the total number of rows
total_rows = len(df)

# Iterate through the DataFrame row by row and process it
for idx, row in tqdm(df.iterrows(), total=total_rows):
    headline = row['headline']
    masked_headline= mask_entities_gliner(headline)
    
    # Append the masked headline to the list
    masked_headlines.append(masked_headline)

    # Print progress every 10% of total rows processed
    if (idx + 1) % (total_rows // 10) == 0:
        print(f"Processed {((idx + 1) / total_rows) * 100:.1f}% of rows")

# Add the masked headlines to the DataFrame
df['maskedHeadline'] = masked_headlines

 10%|▉         | 271/2719 [01:16<11:02,  3.69it/s]

Processed 10.0% of rows


 20%|█▉        | 542/2719 [02:31<09:51,  3.68it/s]

Processed 19.9% of rows


 30%|██▉       | 813/2719 [03:46<08:28,  3.75it/s]

...

 90%|████████▉ | 2439/2719 [11:24<01:17,  3.61it/s]

Processed 89.7% of rows


100%|█████████▉| 2710/2719 [12:42<00:02,  3.75it/s]

Processed 99.7% of rows


100%|██████████| 2719/2719 [12:44<00:00,  3.56it/s]


In [90]:
print("hi")

hi


In [91]:
df["maskedHeadline"]

0       ORGANIZATION books ORGANIZATION, 3 PERSON for ...
1       ORGANIZATION denies wrongdoing amid allegation...
2               Buy ORGANIZATION; target of MONEY: PERSON
3         ORGANIZATION net profit surges to MONEY in DATE
4       ORGANIZATION Q3 net profit down PERCENTAGE to ...
                              ...                        
2714    Reduce ORGANIZATION; target of MONEY: ORGANIZA...
2715      Buy ORGANIZATION; target of MONEY: ORGANIZATION
2716    L&amp;amp;T manufactures PRODUCT for refinery ...
2717    ORGANIZATION Q4 Preview: EVENT, slow demand, E...
2718    ORGANIZATION DATE MONEY falls PERCENTAGE to MO...
Name: maskedHeadline, Length: 2719, dtype: object

In [92]:
df.to_csv('updated_dataframe.csv', index=False)